In [ ]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled, filename = [2005, 2021, False, True, 'df-light.pkl']
else:
    start_year, end_year, chk, sampled, filename = [2005, 2021, True, False, 'df-full.pkl']

from sklearn.model_selection import train_test_split

df = pd.read_pickle(f'./{filename}')
data = df.iloc[:, 1:]
target = df['grav']

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from category_encoders import TargetEncoder, OneHotEncoder
from imblearn.over_sampling import SMOTE, RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.metrics import classification_report_imbalanced
import time

start_time = time.time()

cols_target_encoded = []
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)
scaler = StandardScaler()

encoder_target = TargetEncoder(cols=cols_target_encoded)
encoder_onehot = OneHotEncoder(cols=cols_onehot_encoded)
sampler        = SMOTE(random_state=42)
# sampler        = RandomUnderSampler()
# sampler        = RandomOverSampler()
model          = RandomForestClassifier()


X_train_te = encoder_target.fit_transform(X_train, y_train)
X_test_te  = encoder_target.transform(X_test)

X_train_oh = encoder_onehot.fit_transform(X_train_te, y_train)
X_test_oh  = encoder_onehot.transform(X_test_te)

X_train_sc = scaler.fit_transform(X_train_oh)
X_test_sc  = scaler.transform(X_test_oh)

X_train_rs, y_train_rs = sampler.fit_resample(X_train_sc, y_train)
X_test_rs, y_test_rs   = sampler.fit_resample(X_test_sc, y_test)

params = {
    'n_estimators' : [20, 50], #, 70],
    'criterion' : ('gini', 'entropy'),
    # 'max_depth' : (None, 10, 50),
    # 'min_samples_split' : (2,4,6)
}

grid = GridSearchCV(estimator=model, param_grid=params, cv = 3, verbose=2)
grid.fit(X_train_rs, y_train_rs)

print('Best score  : ', grid.best_score_)
print('Best params : ', grid.best_params_)

y_pred = grid.predict(X_test_rs)
print(classification_report_imbalanced(y_test_rs, y_pred))

print("--- performed in %s seconds ---" % (time.time() - start_time))

In [ ]:
feats = {}
for feature, importance in zip(X_train_oh.columns, grid.best_estimator_.feature_importances_):
    feats[feature] = importance

importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})

# 8 variables les plus importantes
importances.sort_values(by='Gini-importance', ascending=False).head(20)

In [ ]:
from shapash import SmartExplainer

X_test_shap = pd.DataFrame(data=X_test_rs, columns=scaler.get_feature_names_out())
X_test_shap = X_test_shap.iloc[1:3, :]

xpl = SmartExplainer(
    model=grid.best_estimator_,
    # features_dict=house_dict,  # Optional parameter
    preprocessing=encoder, # Optional: compile step can use inverse_transform method
    # postprocessing=postprocess # Optional: see tutorial postprocessing
)

xpl.compile(
    x=X_test_shap
    # y_pred=y_pred, # Optional: for your own prediction (by default: model.predict)
    # y_target=yTest, # Optional: allows to display True Values vs Predicted Values
)

app = xpl.run_app()
